In [18]:
import streamlit as st
import time
import numpy as np
import altair as alt

import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
import os
import pandas as pd
from pathlib import Path

import copy

In [23]:
R_SNOFLK_USER = os.getenv('R_SNOFLK_USER')

@st.experimental_singleton
def get_session():
    
    snowflake.connector.paramstyle = 'pyformat'

    conn = snowflake.connector.connect(
        user=R_SNOFLK_USER,
        password='N/A',
        account='ssq.canada-central.azure',
        authenticator='externalbrowser'
        )

    conn_cs = conn.cursor()
    return conn_cs

cs = get_session()

2022-12-20 21:27:46.247 Snowflake Connector for Python Version: 2.8.3, Python Version: 3.10.8, Platform: Windows-10-10.0.19044-SP0
2022-12-20 21:27:46.248 This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [3]:
date_debut = '2021-12-31'
date_fin = '2022-11-27'
date_debut_vie_ssq = 20211231
date_fin_vie_ssq = 20221127

In [26]:
@st.experimental_singleton
def query_epargne():

    cs.execute('USE DATABASE M_ASSURANCE_INDIVIDUELLE')
    cs.execute('USE SCHEMA ACCES_BII_DATM')

    query_LC_epargne = """\
    SELECT  PRODC.PRODC_NUMBR AS NO_OIPA,
            sum(vente.MNT_NOVL_ARGNT + vente.MNT_NOVL_RVD_EMIS) AS ENTREES
            
    FROM    VENTE

    JOIN PRODC ON VENTE.ID_CONS=PRODC.ID_PRODC

    WHERE   date(DATE_PROD) > %(debut)s AND
            date(DATE_PROD) < %(fin)s
            
    GROUP BY PRODC.PRODC_NUMBR;"""

    cs.execute(query_LC_epargne, {"debut": date_debut, "fin": date_fin})

    df_epargne = pd.DataFrame(cs.fetch_pandas_all())

    return df_epargne

df_epargne_LC = copy.deepcopy(query_epargne())

2022-12-20 21:30:05.071 query: [USE DATABASE M_ASSURANCE_INDIVIDUELLE]
2022-12-20 21:30:06.063 query execution done
2022-12-20 21:30:06.067 Number of results in first chunk: 1
2022-12-20 21:30:06.072 query: [USE SCHEMA ACCES_BII_DATM]
2022-12-20 21:30:06.262 query execution done
2022-12-20 21:30:06.262 Number of results in first chunk: 1
2022-12-20 21:30:06.263 query: [SELECT  PRODC.PRODC_NUMBR AS NO_OIPA, sum(vente.MNT_NOVL_ARGNT + vente.MNT_NOVL_...]
2022-12-20 21:30:25.745 query execution done
2022-12-20 21:30:25.748 Number of results in first chunk: 3001


In [21]:
cs

In [32]:
df_epargne_LC["NO_OIPA"]

0       PCZ-129888
1       CBC-123843
2       C9J-124686
3       PR5-660068
4       OSR-129367
           ...    
4376    WSX-136711
4377    V0W-136002
4378    OLY-129196
4379    W3X-136991
4380    XIU-137364
Name: NO_OIPA, Length: 4381, dtype: object